In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from pandas import read_csv 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt


In [9]:
#DATA PREPROCESSING

#Load data
#data = read_csv('/Users/mariacarolina/Documents/IST/PYFUME_TUTORIAL/DATA/hairdryer.csv', header=None)

data = pd.read_csv('/Users/mariacarolina/Documents/IST/PYFUME_TUTORIAL/DATA/hairdryer.csv',header=None, names=["Voltage", "Temperature"]) 
var_names = data.columns.tolist()[:-1]



#X = data[["Voltage"]]  # X is a DataFrame
#y = data["Temperature"]  # y is a Series

#Convert dataframe to a numpy array
data = data.to_numpy()

# Separate features (X) and target (y)
X = data[:, 0]  # 'input voltage'
y = data[:, 1]  # 'temperature'

# Reshape X as it's a single feature
X = X.reshape(-1, 1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X)


[[6.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]


In [10]:
#Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')
print(f"Partition matrix shape: {part_matrix.shape}")  # Should be (n_samples, nr_clusters)


#Just for visualization purposes
#cluster_labels = np.argmax(part_matrix, axis=1) #Assign each point to the cluster with the highest membership

#plot_data = pd.DataFrame({
    #'Input Voltage': X_train.flatten(),  # Flatten to 1D for DataFrame
    #'Output Temperature': y_train,
    #'Cluster Label': cluster_labels
#})

#average_data = plot_data.groupby('Cluster Label').mean().reset_index() #Calculate average output temperature for each cluster

#Line plot for each cluster
#plt.figure(figsize=(12, 8))
#for cluster in average_data['Cluster Label']:
    #cluster_data = plot_data[plot_data['Cluster Label'] == cluster]
    #plt.plot(cluster_data['Input Voltage'], cluster_data['Output Temperature'], 
     #        marker='o', linestyle='-', label=f'Cluster {cluster}')

#Labels
#plt.title('Cluster Analysis: Input Voltage vs Output Temperature')
#plt.xlabel('Input Voltage')
#plt.ylabel('Output Temperature')
#plt.legend(title='Clusters')
#plt.grid()
#plt.show()


Partition matrix shape: (800, 10)


In [11]:
# Estimate membership functions parameters 
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()
print(antecedent_params)


[('gauss', array([3.0182522 , 1.58680259])), ('gauss', array([6.79022091, 1.56145607])), ('gauss', array([3.02664742, 1.56832355])), ('gauss', array([3.02560148, 1.57062025])), ('gauss', array([6.78813832, 1.55689662])), ('gauss', array([6.78917909, 1.55917446])), ('gauss', array([3.0171977 , 1.58913023])), ('gauss', array([3.02035686, 1.58216025])), ('gauss', array([6.79022091, 1.56145607])), ('gauss', array([3.02245766, 1.57753296]))]


In [12]:
# Estimate consequent parameters
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms(global_fit=True, df=0.0)
print(conseq_params)


[[-14715.3945458   50183.48144141]
 [  -522.243415     3353.45429581]
 [ -1638.60325864   5593.54566567]
 [  7956.07882804 -27124.76362732]
 [ -1063.45428336   6821.23522543]
 [   422.27188069  -2702.99664704]
 [  3910.85893302 -13332.52007438]
 [ 22654.93554821 -77248.8675601 ]
 [  1161.39548505  -7439.32575552]
 [-18134.6317824   61844.05394728]]


In [ ]:
# %% Build first-order Takagi-Sugeno model
modbuilder = SugenoFISBuilder(
    antecedent_sets=antecedent_params, 
    consequent_parameters=conseq_params, 
    variable_names=var_names)

model = modbuilder.get_model()

In [ ]:
# %% Get model predictions
modtester = SugenoFISTester(model, X_test, var_names)
y_pred = modtester.predict()[0]

In [ ]:
# %% Compute regression metrics
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse))
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape*100))
exp_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var))